# Disruption Lab x PowerBox Fa24 Demo

#### Goal: Develop a pipeline that simplifies the process of model development and execution for energy metric forecasting through GCP and Vertex AI.

This notebook achieves this goal by breaking down the issue into four subcategories:
* Automate data acquisition and transformation.
* Run a metric estimation such as future energy forecast.
* Automatically store results in a database.
* Have some mechanism to configure automation and frequency.

### Connecting to the PowerBox Database

Establish a GCP connection with the PowerBox DB. This allows us to query for data using SQL through SQLAlchemy. Also prints a message verifying that we've connected successfully.

In [1]:
%%capture
!pip install git+https://github.com/GoogleCloudPlatform/cloud-sql-python-connector.git
!pip install pystan
!pip install prophet
!pip install pg8000

In [52]:
from google.cloud import storage
import os

def get_key():
    BUCKET_NAME = 'powerbox-test-438421-us-notebooks'
    KEY_PATH = 'key.json' # <BUCKET_NAME>/<KEY_PATH> on Cloud Storage

    LOCAL_PATH = "/tmp/key.json"  # Store the key in a temporary directory
    
    storage_client = storage.Client()
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(KEY_PATH)
    blob.download_to_filename(LOCAL_PATH)
    print(f"Downloaded service account key to {LOCAL_PATH}")

    # Set the GOOGLE_APPLICATION_CREDENTIALS environment variable
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = LOCAL_PATH

# Call this function before making any authenticated requests
get_key()

Downloaded service account key to /tmp/key.json


In [53]:
import os
from sqlalchemy import create_engine, text
from google.cloud.sql.connector import Connector
import google.auth
import pandas as pd
from prophet import Prophet


DB_NAME="santa_ana_cc"
GCP_CONNECTION_NAME="demos-406119:us-central1:pbox-demo-replica"
GOOGLE_APPLICATION_CREDENTIALS= os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
DB_USER="demo_user"
DB_PASS="0EtSv:{pn0hCCTKV"

def get_gcp_connector():
    connector = Connector()
    gcp_connector = connector.connect(
        GCP_CONNECTION_NAME,
        "pg8000",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME,

    )
    return gcp_connector


# create an engine to the postgresql server, with no reference to the database
engine = create_engine("postgresql+pg8000://", creator=get_gcp_connector)
print(engine)

# Test the connection by running a simple query
def test_connection():
    try:
        with engine.connect() as connection:
            # Run a simple query
            result = connection.execute(text("SELECT NOW()"))
            # Fetch and print the result
            for row in result:
                print(f"Connected successfully. Current database time: {row[0]}")
    except Exception as e:
        print(f"Connection failed: {e}")

# Call the test function
test_connection()

Engine(postgresql+pg8000://)
Connected successfully. Current database time: 2024-12-18 22:55:46.353205+00:00


### SQL Query Wrapper

Takes in a SQL query and executes it through SQLAlchemy.

In [54]:
def execute_sql_query(query): # modified to return a list
    try:
        with engine.connect() as connection:
            list = []
            result = connection.execute(text(query))
            # Fetch all results if it's a SELECT query
            if query.strip().upper().startswith("SELECT"):
                rows = result.fetchall()
                for row in rows:
                    list.append(row)
                return list
            else:
                print("Query executed successfully.")
    except Exception as e:
        print(f"Error occurred: {e}")

### Retrieve Metric Data Within A Defined Timeframe

Given a metric, table, starting date, and ending date, retrieve the available data from the PowerBox DB and convert it into a Pandas Dataframe.

In [55]:
def find_df_in_range(metric: str, table: str, start_date: str, end_date: str) -> pd.DataFrame:
    sql_query = 'SELECT ' + metric + ', timestamp FROM "' + table + '" WHERE "timestamp" BETWEEN ' + start_date + ' AND ' + end_date
    
    results = execute_sql_query(sql_query)
    measurements = [row[0] for row in results]
    timestamps = [pd.Timestamp(row[1]) for row in results]
    
    df = pd.DataFrame({"time": timestamps, metric: measurements})
    df['time'] = pd.to_datetime(df['time'], format='%Y/%m/%d %H:%M:%S.%f', errors='coerce')
    df['time'] = df['time'].dt.floor('s')
    df = df.dropna(subset=['time'])
    df = df.rename(columns={"time": "ds", metric: "y"})
    return df

In [56]:
metric = 'real_power'
table = 'super_meter_1.1'
start_date = "'2024-03-07'"
end_date = "'2024-04-07'"
test = find_df_in_range(metric, table, start_date, end_date)
print(test)

                        ds                   y
0      2024-03-07 00:00:02           37748.658
1      2024-03-07 00:00:08           38080.832
2      2024-03-07 00:00:14           37795.834
3      2024-03-07 00:00:21           37807.625
4      2024-03-07 00:00:27  37904.956000000006
...                    ...                 ...
385344 2024-04-06 23:59:27           29241.553
385345 2024-04-06 23:59:34            29422.07
385346 2024-04-06 23:59:40           30411.737
385347 2024-04-06 23:59:47  28934.063000000002
385348 2024-04-06 23:59:53  28911.061999999998

[385349 rows x 2 columns]


### Metric Forecasting and Models

Use Prophet to create a model forecasting energy metrics over time. **Notice that running predictions with large amounts of data requires a lot more time and memory**.

In [57]:
m = Prophet(changepoint_prior_scale=0.01).fit(test)

22:56:26 - cmdstanpy - INFO - Chain [1] start processing
23:00:17 - cmdstanpy - INFO - Chain [1] done processing


In [58]:
future = m.make_future_dataframe(periods=24*7, freq='h')
future.tail()
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
385512,2024-04-13 19:59:53,52006.568821,10716.516645,96088.259400
385513,2024-04-13 20:59:53,50835.999939,9279.984767,94259.559700
385514,2024-04-13 21:59:53,47057.083465,3219.258114,89680.261160
385515,2024-04-13 22:59:53,42840.928877,382.893661,88984.370134
385516,2024-04-13 23:59:53,40314.747481,-4090.453314,89334.240435


The following graph predicts **'real_power' over time**. It uses the above 'periods' and 'frequency' arguments in 'make_future_dataframe' to make the prediction cycle over a 24-hour period.\
Notice the fluctuations in our forecast. Our model captures the cyclic nature of energy consumption over a day-night period, where a business may be consuming more energy during the day than at night.

In [ ]:
real_power_forecast = m.plot(forecast)

The following code processes and visualizes the predictions made by the Prophet model.\
Calculates the mean of predicted values (yhat) over fixed intervals (7000 data points, corresponding to day and night cycles).\
The bar chart visualizes the distribution of these means, helping to capture the temporal trends and variability of predictions over consistent time periods, and offering insights into patterns in forecasted power consumption.

In [ ]:
import matplotlib.pyplot as plt 
forecast_table = forecast[['ds', 'yhat']]

interval_size = 7000

means = [
    forecast_table['yhat'][i:i + interval_size].mean()
    for i in range(0, len(forecast_table), interval_size)
]

interval_indices = range(len(means))
plt.figure(figsize=(10, 6))
plt.bar(interval_indices, means, width=0.8, color='blue', alpha=0.7)
plt.title("Distribution of Means for the yhat")
plt.xlabel("Interval Index")
plt.ylabel("Mean of yhat")
plt.show()

### Analyzing And Visualizing Current Data

In [61]:
# function that changes the SQL queries into a Pandas dataframe
def execute_pandas_query(query):
    try:
        with engine.connect() as connection:
            result = connection.execute(text(query))
            # Convert result to DataFrame
            if query.strip().upper().startswith("SELECT"):
                df = pd.DataFrame(result.fetchall(), columns=result.keys())
                return df
            else:
                print("Query executed successfully.")
                return None
    except Exception as e:
        print(f"Error occurred: {e}")
        return None

In [62]:
%%capture
sql_query = 'SELECT real_power, timestamp FROM "super_meter_1.1" WHERE "timestamp" BETWEEN ' + "'2024-03-07'" + ' AND ' + "'2024-04-07'"

In [63]:
df = execute_pandas_query(sql_query)
df.set_index('timestamp', inplace=True)
df

,real_power
timestamp,
2024-03-07 00:00:02.028179,37748.658
2024-03-07 00:00:08.706194,38080.832
2024-03-07 00:00:14.849997,37795.834
2024-03-07 00:00:21.199028,37807.625
2024-03-07 00:00:27.752682,37904.956000000006
...,...
2024-04-06 23:59:27.359311,29241.553
2024-04-06 23:59:34.118671,29422.07
2024-04-06 23:59:40.672097,30411.737


### Real Power Chart

Visualizes the 'real_power' metric data that we've collected over the selected timeframe.

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['real_power'], name='power'))
fig.show()

In [65]:
%%capture
sql_query2 = 'SELECT timestamp, voltage_a, voltage_b, voltage_c FROM "super_meter_1.1" WHERE "timestamp" BETWEEN ' + "'2024-03-07'" + ' AND ' + "'2024-04-07'"

In [66]:
df2 = execute_pandas_query(sql_query2)
df2.set_index('timestamp', inplace=True)
df2

,voltage_a,voltage_b,voltage_c
timestamp,,,
2024-03-07 00:00:02.028179,272.3,274.4,276.0
2024-03-07 00:00:08.706194,272.4,274.4,275.8
2024-03-07 00:00:14.849997,272.1,274.5,275.8
2024-03-07 00:00:21.199028,272.2,274.4,275.9
2024-03-07 00:00:27.752682,271.9,274.2,275.6
...,...,...,...
2024-04-06 23:59:27.359311,273.9,274.5,276.9
2024-04-06 23:59:34.118671,274.1,275.0,277.0
2024-04-06 23:59:40.672097,273.1,273.7,275.9


### Voltage Comparison Chart

Visualizes the various voltage metric data that we've collected over the selected timeframe.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df2.index, y=df2['voltage_a'], name='voltage_a'))
fig.add_trace(go.Scatter(x=df2.index, y=df2['voltage_c'], name='voltage_c'))
fig.add_trace(go.Scatter  (x=df2.index, y=df2['voltage_b'], name='voltage_b'))
fig.show()

In [68]:
# df['timestamp'] = pd.to_datetime(df['timestamp'])
# df.set_index('timestamp', inplace=True)

# Adjusting the resampling frequency to '1H' (hourly) instead of 'D' (daily)
new_df = df['real_power'].resample('1h').sum() / 3600 / 1000  # Adjust for kWh if needed
new_df = new_df.rename("energy")

# Convert to DataFrame and print the results to check
new_df = new_df.to_frame()
new_df

,energy
timestamp,
2024-03-07 00:00:00,6.638209520833333332777777778
2024-03-07 01:00:00,5.915694245833333336111111111
2024-03-07 02:00:00,6.365566577777777773888888889
2024-03-07 03:00:00,5.908588094722222256388888889
2024-03-07 04:00:00,5.607316477777777769444444444
...,...
2024-04-06 19:00:00,6.235854179444444453611111111
2024-04-06 20:00:00,6.936505670277777760277777778
2024-04-06 21:00:00,6.434681016666666666111111111


### Hourly Energy Consumption Chart

Visualizes the 'real_power' metric data that we've collected over the selected timeframe in an hourly breakdown.

In [ ]:
import plotly.express as px

fig = px.bar(new_df, x=new_df.index, y='energy', title="Hourly Energy Consumption")
fig.show()

In [70]:
hourly_energy = df['real_power'].groupby(pd.Grouper(freq='1h')).sum()/3600/1000
frame = { 'ds': hourly_energy.index, 'y': hourly_energy }
df_new = pd.DataFrame(frame)
df_new

,ds,y
timestamp,,
2024-03-07 00:00:00,2024-03-07 00:00:00,6.638209520833333332777777778
2024-03-07 01:00:00,2024-03-07 01:00:00,5.915694245833333336111111111
2024-03-07 02:00:00,2024-03-07 02:00:00,6.365566577777777773888888889
2024-03-07 03:00:00,2024-03-07 03:00:00,5.908588094722222256388888889
2024-03-07 04:00:00,2024-03-07 04:00:00,5.607316477777777769444444444
...,...,...
2024-04-06 19:00:00,2024-04-06 19:00:00,6.235854179444444453611111111
2024-04-06 20:00:00,2024-04-06 20:00:00,6.936505670277777760277777778
2024-04-06 21:00:00,2024-04-06 21:00:00,6.434681016666666666111111111


### Hourly Energy Consumption Forecast

Forecasting 'real_power' metric in an hourly format.

In [ ]:
m = Prophet(changepoint_prior_scale=0.01).fit(df_new)

future = m.make_future_dataframe(periods=24*7, freq='h')
forecast = m.predict(future)

fig1 = m.plot(forecast, xlabel = 'timestamp',ylabel = 'energy')
ax = fig1.gca()
ax.set_title("Hourly Energy Consumption", size=24)

## Storage and Scheduling

#### Procedure:
Click 'Execute' in the tabs above to open the executor. Here, we can schedule automated notebook runs, and store the outputs in a Cloud Storage Bucket.

Using the Unix Cron format, we can input the following schedule: */15 * * * *


This schedules the notebook to run every 15 minutes, every hour, every day of the month, every month, every day of the week.
Make sure to store the results in a Cloud Storage bucket (advanced options).

#### Set up the client

In [72]:
from google.cloud import storage

# set up client and bucket
BUCKET_NAME = 'powerbox-test-438421-us-notebooks'
OUTPUT_PATH = 'scheduled_results_real' # output to <BUCKET_NAME>/<OUTPUT_PATH> in Cloud Storage

client = storage.Client()
bucket = client.bucket(BUCKET_NAME)

#### Uploading output

We want to make sure that our outputs are distinguishable in some way so that we prevent overwriting existing outputs, and for easy identification of past results. Use a timestamp to identify each output.

Example for uploading plots. We can also save the data in text, as a PNG, CSV, etc.

In [73]:
import datetime
import os
import matplotlib.pyplot as plt

# generate a timestamp
timestamp = datetime.datetime.now().strftime("%Y_%m_%d_%H:%M:%S")
print(timestamp)

# create the output file with the timestamp
output_file = f"real_power_forecast_{timestamp}.png" # png
print(output_file)

# write plot to output_file
real_power_forecast.savefig(output_file) # real_power_forecast model calculated above

# save and upload to Cloud Storage
blob = bucket.blob(OUTPUT_PATH + "/" +  output_file) # store at <BUCKET_NAME>/<OUTPUT_PATH>/<output_file> in Cloud Storage
blob.upload_from_filename(output_file)
print(f"Uploaded {output_file} to {BUCKET_NAME}/{OUTPUT_PATH}")

# remove local output
os.remove(output_file)

2024_12_18_23:06:57
real_power_forecast_2024_12_18_23:06:57.png
Uploaded real_power_forecast_2024_12_18_23:06:57.png to powerbox-test-438421-us-notebooks/scheduled_results_real
